# Base dataset preparation

This notebook preparers the "base" dataset, i.e., a collection of properly annotated 5'UTRs.
One can use this dataset with `uBERTa_loader` to prepare the training/validation/testing data for the model.

In [1]:
from collections import Counter
from itertools import chain, starmap
from pathlib import Path

import numpy as np
import pandas as pd
from more_itertools import sliding_window
from toolz import curry
from tqdm.auto import tqdm
from uBERTa.base import VALID_START
from uBERTa.utils import Ref, pBWs, reverse_complement

In [2]:
DATA = Path('../data')
DATA.mkdir(exist_ok=True)

## Expected outputs
- DS_BASE.tsv          -- base dataset with annotated 5'UTRs
- dataset_labeling.tsv -- labeling of the analyzed genes as training, validation, and testing

## Initial data

This notebook requires:
- hg38 reference genome
- ribo-seq experimental signal from GWIPS-viz (we use P-site identification experiments only)
- our hand-crafted dataset as a bed file
- 5'UTR regions: provided with the rest of the data; also can be obtained via R package [ORFik](https://bioconductor.org/packages/release/bioc/html/ORFik.html) (see `extract_5UTR.rmd` notebook)
- A mapping between Ensembl gene IDs and OMIM gene names
- A list of analyzed genes

Download the data and unpack it into the `DATA` dir initialized above.

There are two archives you'll need:
1. [prepare_base_dataset.zip](https://drive.google.com/file/d/1phgab69jDsvgMqOeGUp9mGdDDRU12pk_/view?usp=sharing) (riboseq data, 5'UTRs, etc.) 
2. [hg38.fa.tar.gz](https://drive.google.com/file/d/1obZdHGf06FFeGw7PCDh5gvMtRjHLKMHu/view?usp=sharing) (reference and its indexing in FASTA format)

Download both of them and unpack into `DATA`. The expected structure of the `DATA` after this would be:

```
|____ENSG2OMIM.csv                                                                         
|____List_of_analysed_unique_genes.csv
|____hg38.fa
|____hg38.fa.fai
|____All_starts_AD+AR_v2.bed
|____p-sites
| |____Raj16_All.RiboProInit.bw
| |____Fijalkowska17_All.RiboProInit.bw
| |____Gao14_All.RiboProInit.bw
| |____Ji15_All.RiboProInit.bw
| |____Zhang17_All.RiboProInit.bw
| |____Chen20_All.RiboProInit.bw
| |____Gawron16_All.RiboProInit.bw
| |____Crappe15_All.RiboProInit.bw
|____uORF_search_space
| |____uORF_search_space_cage_genes.gff
| |____uORF_search_space_cage_transcripts.gff
```

For instance, starting from the project's root.

1. Download the data

```bash
gdown --fuzzy https://drive.google.com/file/d/1obZdHGf06FFeGw7PCDh5gvMtRjHLKMHu/view?usp=sharing
gdown --fuzzy https://drive.google.com/file/d/1phgab69jDsvgMqOeGUp9mGdDDRU12pk_/view?usp=sharing
```

2. Unpack downloads

```bash
unzip prepare_base_dataset.zip
rm prepare_base_dataset.zip

for path in $(ls *.tar.gz); do
    tar -xzf $path
done

rm *.tar.gz

ls
```

-> 

```bash
All_starts_AD+AR_v2.bed  ENSG2OMIM.csv  hg38.fa  hg38.fa.fai  List_of_analysed_unique_genes.csv  p-sites  uORF_search_space
```

In [3]:
# Reference genome to fetch sequence regions
ref = Ref(DATA / 'hg38.fa')
# Experimental ribo-seq signal as a collection of BigWig files queried simultaneously
pbws = pBWs(Path(DATA / 'p-sites').glob('*'))

## Parse the hand-crafted dataset

- Convert the hand-crafted dataset into a dataframe
- Validate annotation correctness
- Fetch start codon sequences
- Filter invalid start codons (should be removed in the final version, but exercising some caution never hurts)

In [6]:
def parse_hand_crafted(path):
    """
    Convert a bed file with hand-crafted uORFs to a Pandas dataframe
    """
    df = pd.read_csv(
        path, sep='\s+', skiprows=1, skipfooter=1,
        names=['Chrom', 'Start', 'End', 'Ann', 'X', 'Strand'])
    valid_ann_idx = np.array(
        [len(x.split('-')) == 5 for x in df['Ann']])
    df_invalid = df[~valid_ann_idx]
    df = df[valid_ann_idx]
    df[['Group', 'Gene', 'StartCodon', 'KozakScore', 'Level']] = [
        x.split('-') for x in df['Ann']]
    return df, df_invalid

In [7]:
ds, ds_inv = parse_hand_crafted(
    DATA / 'All_starts_AD+AR_v2.bed'
)
ds = ds.drop(
    columns=['Ann', 'X', 'KozakScore']
).sort_values(
    ['Chrom', 'Start']
)

print(len(ds), len(ds_inv))
ds.tail()

7312 0


/tmp/ipykernel_24783/3068235612.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(


,Chrom,Start,End,Strand,Group,Gene,StartCodon,Level
5511,chrX,155264457,155264460,-,u,RAB39B,ATC,88
7284,chrX,155264493,155264496,-,u,RAB39B,ATG,113
7058,chrX,155545273,155545276,-,m,TMLHE,ATG,0
5512,chrX,155612830,155612833,-,u,TMLHE,CTG,247
5513,chrX,155612861,155612864,-,u,TMLHE,CTG,127


In [8]:
ds['StartCodonFetched'] = [
    ref.fetch(*x[1:]).upper() for x in
    tqdm(ds[['Chrom', 'Start', 'End']].itertuples(), total=len(ds))]

neg_idx = ds['Strand'] == '-'
ds.loc[neg_idx, 'StartCodonFetched'] = ds.loc[
    neg_idx, 'StartCodonFetched'].apply(reverse_complement)

  0%|          | 0/7312 [00:00<?, ?it/s]

In [9]:
print(f'Initial {len(ds)}')
ds = ds[ds.StartCodon == ds.StartCodonFetched]
print(f'Matching start codons: {len(ds)}')
ds = ds.drop(columns=['StartCodonFetched'])
ds = ds[ds.StartCodon.isin(VALID_START)]
print(f'Supported start codons: {len(ds)}')

Initial 7312
Matching start codons: 7303
Supported start codons: 7296


Mapping gene names to ensemble IDs

In [10]:
name2id = dict(
    x[1:] for x in
    pd.read_csv(DATA / 'ENSG2OMIM.csv', sep=';').itertuples())
ds['GeneID'] = ds['Gene'].map(name2id.get)

In [11]:
print(f'Final ds size: {len(ds)}')
ds.head()

Final ds size: 7296


,Chrom,Start,End,Strand,Group,Gene,StartCodon,Level,GeneID
3028,chr1,1013523,1013526,+,u,ISG15,TTG,770,ENSG00000187608
5514,chr1,1013546,1013549,+,ma,ISG15,GTG,556,ENSG00000187608
6474,chr1,1013573,1013576,+,m,ISG15,ATG,0,ENSG00000187608
5920,chr1,1020172,1020175,+,m,AGRN,ATG,174,ENSG00000188157
2302,chr1,1349062,1349065,-,m,DVL1,ATG,379,ENSG00000107404


## Parse 5'UTR regions

- Parse GFF files with 5'UTR coordinates obtained externally using ORFik
- Fetch and enumerate sequences
- Fetch experimental signal for 5'UTRs
- Concatenate data transcript-wise
- Handle strand direction

In [12]:
def get_anno_value(field_name, anno):
    _anno = dict(x.split('=') for x in anno.split(';'))
    return _anno[field_name]

def parse_granges_gff(path):
    allowed_chr = list(map(str, range(1, 23))) + ['X', 'Y']

    df_gff = pd.read_csv(
        path, usecols=[0, 3, 4, 6, 8], sep=r'\s+', low_memory=False,
        names=['Chrom', 'Start', 'End', 'Strand', 'Anno'], skiprows=3)
    print(f'Initial size: {len(df_gff)}')
    df_gff = df_gff[df_gff.Anno.apply(lambda x: 'exon' in x)]
    print(f'Filtered to exons: {len(df_gff)}')
    df_gff = df_gff[df_gff.Chrom.isin(allowed_chr)]
    print(f'Filtered to canonical chromosomes: {len(df_gff)}')
    df_gff['Chrom'] = df_gff.Chrom.apply(lambda x: 'chr' + x)
    df_gff['ExonID'] = df_gff.Anno.apply(lambda x: get_anno_value('exon_name', x))
    df_gff['ID'] = df_gff.Anno.apply(lambda x: get_anno_value('Name', x))
    df_gff = df_gff.drop(columns=['Anno'])
    df_gff = df_gff.drop_duplicates(ignore_index=True)
    print(f'Filtered out duplicates: {len(df_gff)}')
    return df_gff

@curry
def join(it, sep=';'):
    return sep.join(map(str, it))

def safe_take_fst(vs):
    if len(vs.unique()) != 1:
        raise ValueError(vs)
    return vs.iloc[0]

The two files parsed below encompass exactly the same 5'UTR regions, only the IDs attached to regions are on different level (genes and transcripts, respectively).

In [13]:
genes_path = DATA / 'uORF_search_space' / 'uORF_search_space_cage_genes.gff'
transcripts_path = DATA / 'uORF_search_space' / 'uORF_search_space_cage_transcripts.gff'
assert genes_path.exists()
assert transcripts_path.exists()

In [14]:
df_genes = parse_granges_gff(
    genes_path
).rename(
    columns={'ID': 'GeneID'}
)
df_trans = parse_granges_gff(
    transcripts_path
).rename(
    columns={'ID': 'TranscriptID'}
)

Initial size: 252701
Filtered to exons: 163270
Filtered to canonical chromosomes: 163233
Filtered out duplicates: 112368
Initial size: 252701
Filtered to exons: 163270
Filtered to canonical chromosomes: 163233
Filtered out duplicates: 163233


In [15]:
var = ['Chrom', 'Start', 'End', 'Strand']

df_utr = pd.merge(
    df_genes, df_trans, on=var +['ExonID']
).sort_values(
    var
).reset_index(drop=True)
df_utr['Start'] = df_utr['Start'] - 1
len(df_utr)

163233

In [16]:
# df.to_csv('data/intermediate/5UTR_exons.tsv', sep='\t', index=False)

Parsed genomic ranges are continuous exonic regions within 5'UTR. We first compose sequence-level features for them, and then concatenate these features (sequences, enumeration, signal) for each transcript.

Example (exons are already sorted by the starting coordinate in the ascending order):
```
ENST   ENSE  Seq  Enum    Signal
100500 1     ACGT 1,2,3,4 0.1.0.0,0.1,20.0
100500 2     CCGT 6,7,8,9 0.2.0.1,4.0,0.0
```
-->
```
ENST   Seq      Enum            Signal
100500 ACGTCCGT 1,2,3,4,6,7,8,9 0.1.0.0,0.1,20.0,0.2.0.1,4.0,0.0
```

In [17]:
df_utr['Seq'] = [
    ref.fetch(*x[1:]).upper() for x in 
    tqdm(df_utr[['Chrom', 'Start', 'End']].itertuples(), total=len(df_utr), desc='Fetching seqs')
]
df_utr['SeqEnum'] = [
    join(range(row['Start'], row['End']), sep=',') for _, row in 
    tqdm(df_utr.iterrows(), total=len(df_utr), desc='Enumerating seqs')
]
df_utr['Signal'] = [
    join(pbws.query(*x[1:]), sep=',') for x in 
    tqdm(df_utr[['Chrom', 'Start', 'End']].itertuples(), total=len(df_utr), desc='Fetching signal')
]

Fetching seqs:   0%|          | 0/163233 [00:00<?, ?it/s]

Enumerating seqs:   0%|          | 0/163233 [00:00<?, ?it/s]

Fetching signal:   0%|          | 0/163233 [00:00<?, ?it/s]

In [18]:
df_utr = df_utr.groupby(
    'TranscriptID', as_index=False
).agg(
    {'ExonID': join, 
     'GeneID': join, 
     'Seq': join(sep=''), 
     'SeqEnum': join(sep=','),
     'Signal': join(sep=','),
     'Chrom': safe_take_fst,
     'Strand': safe_take_fst
    })

Sanity check: all the concatenated sequences are of equal lengths

In [19]:
all(len(enum.split(',')) == len(signal.split(',')) == len(seq) 
    for _, seq, enum, signal in df_utr[['Seq', 'SeqEnum', 'Signal']].itertuples())

True

Check the distributions of (1) the number of exons per transcript, and (2) the 5'UTR exonic sequence's length.

In [21]:
reverse_join = lambda x: join(x.split(',')[::-1], sep=',')
idx = df_utr.Strand == '-'
df_utr.loc[idx, 'Seq'] = df_utr.loc[idx, 'Seq'].apply(reverse_complement)
df_utr.loc[idx, 'SeqEnum'] = df_utr.loc[idx, 'SeqEnum'].apply(reverse_join)
df_utr.loc[idx, 'Signal'] = df_utr.loc[idx, 'Signal'].apply(reverse_join)

In [22]:
df_utr.head()

,TranscriptID,ExonID,GeneID,Seq,SeqEnum,Signal,Chrom,Strand
0,ENST00000000233,ENSE00001872691,ENSG00000004059,CTGCTGCTGCTGCGCCCCATCCCCCCGCGGCCGGCCAGTTCCAGCC...,"127588410,127588411,127588412,127588413,127588...","110.0,9.0,28.0,65.0,1.0,15.0,52.0,1.0,34.0,71....",chr7,+
1,ENST00000000412,ENSE00003523177;ENSE00001348389,ENSG00000003056;ENSG00000003056,AGAGTGGGGCACAGCGAGGCGCTAGGGGGAACGCTGGCCTCTGAAA...,"8949644,8949643,8949642,8949641,8949640,894963...","2.0,19.0,68.0,143.0,14.0,7.0,3.0,1.0,1.0,4.0,0...",chr12,-
2,ENST00000000442,ENSE00001884684;ENSE00001195360,ENSG00000173153;ENSG00000173153,GTCAGCTGGAGGAAGCGGAGTAGGAAGCGGCCGCGATGTCCTTTTG...,"64305523,64305524,64305525,64305526,64305527,6...","0.0,0.0,1.0,4.0,3.0,0.0,1.0,1.0,1.0,9.0,0.0,4....",chr11,+
3,ENST00000001008,ENSE00000802791,ENSG00000004478,CCTACCCCAGCTCTCGCGCCGCGTGCAGAGGTGCTCAAGCCTCCTC...,"2794969,2794970,2794971,2794972,2794973,279497...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3....",chr12,+
4,ENST00000001146,ENSE00000401861,ENSG00000003137,ACAGCCAATCCCCCGAGCGGCCGCCAAC,"72147861,72147860,72147859,72147858,72147857,7...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",chr2,-


## Annotate 5'UTRs

For the 5'UTRs, we assign classes to all supported start codons according to our hand-crafted dataset.
Namely, for each codon, we place:
- 1, if it is within our data
- 0, if it is a valid start codon, but its absent in our data
- 100 if none of the above is True (-100 is a default masking value for the loss functions in Pytorch)

We then append (-100, -100) to classes of each sequence to account for kmerization. 

For example, in the sequence below, where ATG is absent in our data, and CTG is present, we'll compose the following sequence of classes:
```
ATGCTG -> ATG TGC GCT CTG -> 0 -100 -100 1 -100 -100
```
Hence, effectively we attribute a class to each character of a sequence.

In [23]:
def subset_to_positives(row, positives):
    chrom, strand = row['Chrom'], row['Strand']
    positives_enum = ",".join(
        x for x in row['SeqEnum'].split(',') 
        if (chrom, strand, int(x)) in positives)
    return np.nan if not positives_enum else positives_enum

def validate_codon(start, codon, seq, seq_enum):
    """Validate the codon sequence"""
    idx_start = np.where(seq_enum == start)[0]
    seq_codon = seq[idx_start:idx_start + 3]
    return seq_codon == codon

def classify_seq_codons(row):
    def classify_codon(codon, pos):
        if int(pos) in positives:
            return 1
        if codon in VALID_START:
            return 0
        return -100

    kmers3 = map(lambda x: ''.join(x), sliding_window(row.Seq, 3))
    enum = row.SeqEnum.split(',')
    positives = row['SeqEnumPositive']
    if not isinstance(positives, str):
        positives = []
    else:
        positives = list(map(int, positives.split(',')))
        # this accounts for the reversed sequences having 
        # a start codon coordinate at the end of an actual 
        # start codon given the correct direction
        # e.g., for a "-" strand codon ATG with coordinates (3,2,1)
        # our data will have coordinate "1" instead of "3" 
        # (and, for the "+" strand, vice versa). Here, we unify the classes' 
        # placement within a kmer sequence so that it's always assigned 
        # by the first nucleotide of a kmer 
        if row['Strand'] == '-':
            positives = [x + 2 for x in positives]

    labels = starmap(classify_codon, zip(kmers3, enum))

    return ','.join(map(str, chain(labels, [-100, -100])))

def safe_take_fst(vs):
    vs = set(vs.split(';'))
    assert len(vs) == 1
    return vs.pop()

def verify_codons(row):
    classes = np.array(list(map(int, row['Classes'].split(','))))
    m = classes != -100
    starts = np.array(tuple(map(
        lambda x: ''.join(x), sliding_window(row['Seq'], 3))) + ('PAD', 'PAD'))
    return set(starts[m]).issubset(VALID_START)

Compose the "search space" of positive classes. Hence, for a valid start codon to classify as positive, its coordinates (chrom, strand, start)  must by present in our data.

In [24]:
positives = tuple(x[1:] for x in ds[['Chrom', 'Strand', 'Start']].itertuples())
positives[:3]

(('chr1', '+', 1013523), ('chr1', '+', 1013546), ('chr1', '+', 1013573))

Merciless slow and painful yolo strategy... It however, ensures that we assign classes correctly.

In [25]:
df_utr['SeqEnumPositive'] = [
    subset_to_positives(row, positives) for _, row in 
    tqdm(df_utr.iterrows(), total=len(df_utr), desc='Subsetting')]

Subsetting:   0%|          | 0/89404 [00:00<?, ?it/s]

In [26]:
df_utr['Classes'] = [
    classify_seq_codons(row) for _, row in 
    tqdm(df_utr.iterrows(), total=len(df_utr), desc='Assigning classes')]

Assigning classes:   0%|          | 0/89404 [00:00<?, ?it/s]

Next, we verify the labeled start codons. Namely, we use the -100 values of the assigned classes as mask, and check that applying this mask results in subsetting the sequence to valid start codons.

In [27]:
idx = np.array([verify_codons(row) for _, row in tqdm(df_utr.iterrows(), total=len(df_utr))])
print(f'{(~idx).sum()} problematic, {idx.sum()} OK')
df_utr = df_utr[idx]

  0%|          | 0/89404 [00:00<?, ?it/s]

0 problematic, 89404 OK


Frequently, different transcripts don't change the 5'UTR sequence. Below, we get rid of such duplicates by retaining only unique sequences. Thus, the retained sequences will be labeled by the first transcript ID in the dataset.

In [28]:
print(f'Initial size: {len(df_utr)}')
df_utr = df_utr.drop_duplicates('Seq')
print(f'Removed duplcates: {len(df_utr)}')

Initial size: 89404
Removed duplcates: 79677


In [29]:
Counter(chain.from_iterable(
    df_utr.loc[
        ~df_utr.SeqEnumPositive.isna(), 'Classes'
    ].apply(
        lambda x: x.split(','))
))

Counter({'-100': 1967914, '0': 283973, '1': 19572})

Below is another sanity check which verifies that no transcript ID is associated with multiple gene IDs.

In [30]:
df_utr['GeneID'] = df_utr['GeneID'].apply(safe_take_fst)

Finally, we assign boolean values to gene IDs that had been included into the manual curation process. The list of analyzed genes is an external resource. Additionally, since we already marked posititive examples via coordinates, we include the corresponding gene IDs to the list.

In [31]:
with (DATA / 'List_of_analysed_unique_genes.csv').open() as f:
    gene_names = [l.rstrip() for l in f]
    genes_curated = set(name2id[n] for n in gene_names if n in name2id)
genes_ds = set(df_utr.loc[~df_utr['SeqEnumPositive'].isna(), 'GeneID'])
analyzed_genes = genes_curated | genes_ds

print(len(genes_ds - genes_curated), len(genes_curated - genes_ds), len(analyzed_genes))

df_utr['AnalyzedGene'] = False
df_utr.loc[df_utr['GeneID'].isin(analyzed_genes), 'AnalyzedGene'] = True
df_utr['AnalyzedGene'].value_counts()

56 1731 3699


False    60025
True     19652
Name: AnalyzedGene, dtype: int64

In [32]:
df_utr.to_csv(DATA / 'DS_BASE.tsv', sep='\t', index=False)

## Divide the modeling dataset into training, validation, and testing

The modeling dataset encompasses the sequences of analyzed genes. Here, we divide genes into training, validation, and testing subsets and label their sequences accordingly.

In [33]:
df_mod = df_utr[df_utr.AnalyzedGene].copy().reset_index(drop=True)
len(df_mod)

19652

In [34]:
np.random.seed(287)

df_genes = pd.DataFrame({'GeneID': df_mod['GeneID'].unique()})
val_frac, test_frac = 0.1, 0.1
l = len(df_genes)
n_test = int(l * val_frac)
n_val = int(l * test_frac)
n_train = l - n_test - n_val
labels = np.concatenate(
    [np.full(n_train, 'Train'), 
     np.full(n_val, 'Val'), 
     np.full(n_test, 'Test')])
np.random.shuffle(labels)
df_genes['Dataset'] = labels 
df_genes['Dataset'].value_counts()

Train    2948
Test      368
Val       368
Name: Dataset, dtype: int64

In [35]:
df_genes.to_csv(DATA / 'dataset_labeling.tsv', sep='\t', index=False)